#  Test PhotoZ Kernels Cython

In [3]:
import numpy as np
from delight.utils import *
from delight.photoz_kernels_cy import \
    kernelparts, kernelparts_diag, kernel_parts_interp
from delight.utils_cy import find_positions


In [4]:
size = 50
nz = 150
numBands = 2
numLines = 5
numCoefs = 10
relative_accuracy = 0.1

In [5]:
def test_diagonalOfKernels():
    """
    Test that diagonal of kernels and derivatives are correct across functions.
    """
    X = random_X_bzl(size, numBands=numBands)
    X2 = X

    fcoefs_amp, fcoefs_mu, fcoefs_sig = random_filtercoefs(numBands, numCoefs)
    lines_mu, lines_sig = random_linecoefs(numLines)
    var_C, var_L, alpha_C, alpha_L, alpha_T = random_hyperparams()
    norms = np.sqrt(2*np.pi) * np.sum(fcoefs_amp * fcoefs_sig, axis=1)

    NO1, NO2 = X.shape[0], X2.shape[0]
    b1 = X[:, 0].astype(int)
    b2 = X2[:, 0].astype(int)
    fz1 = 1 + X[:, 1]
    fz2 = 1 + X2[:, 1]
    KC, KL \
        = np.zeros((NO1, NO2)), np.zeros((NO1, NO2))
    D_alpha_C, D_alpha_L, D_alpha_z \
        = np.zeros((NO1, NO2)), np.zeros((NO1, NO2)), np.zeros((NO1, NO2))
    kernelparts(NO1, NO2, numCoefs, numLines,
                alpha_C, alpha_L,
                fcoefs_amp, fcoefs_mu, fcoefs_sig,
                lines_mu[:numLines], lines_sig[:numLines], norms,
                b1, fz1, b2, fz2, True,
                KL, KC,
                D_alpha_C, D_alpha_L, D_alpha_z)

    KC_diag, KL_diag\
        = np.zeros((NO1,)), np.zeros((NO1,))
    D_alpha_C_diag, D_alpha_L_diag = np.zeros((NO1,)), np.zeros((NO1,))
    kernelparts_diag(NO1, numCoefs, numLines,
                     alpha_C, alpha_L,
                     fcoefs_amp, fcoefs_mu, fcoefs_sig,
                     lines_mu[:numLines], lines_sig[:numLines], norms,
                     b1, fz1, True, KL_diag, KC_diag,
                     D_alpha_C_diag, D_alpha_L_diag)

    np.testing.assert_almost_equal(KL_diag, np.diag(KL))
    np.testing.assert_almost_equal(KC_diag, np.diag(KC))
    np.testing.assert_almost_equal(D_alpha_C_diag, np.diag(D_alpha_C))
    np.testing.assert_almost_equal(D_alpha_L_diag, np.diag(D_alpha_L))




In [8]:
test_diagonalOfKernels()

In [6]:
def test_find_positions():

    a = np.array([0., 1., 2., 3., 4.])
    b = np.array([0.5, 2.5, 3.0, 3.1, 4.0])
    pos = np.zeros(b.size, dtype=np.long)
    find_positions(b.size, a.size, b, pos, a)
    np.testing.assert_almost_equal(pos, [0, 2, 2, 3, 3])

In [9]:
test_find_positions()

<ipython-input-6-f1afc12af21e>:5: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pos = np.zeros(b.size, dtype=np.long)


In [7]:
def test_kernel_parts_interp():

    fcoefs_amp, fcoefs_mu, fcoefs_sig = random_filtercoefs(numBands, numCoefs)
    lines_mu, lines_sig = random_linecoefs(numLines)
    var_C, var_L, alpha_C, alpha_L, alpha_T = random_hyperparams()
    norms = np.sqrt(2*np.pi) * np.sum(fcoefs_amp * fcoefs_sig, axis=1)

    zgrid = np.linspace(0, 3, num=nz)
    opzgrid = 1 + zgrid

    KC_grid, KL_grid =\
        np.zeros((numBands, numBands, nz, nz)),\
        np.zeros((numBands, numBands, nz, nz))
    D_alpha_C_grid, D_alpha_L_grid, D_alpha_z_grid =\
        np.zeros((numBands, numBands, nz, nz)),\
        np.zeros((numBands, numBands, nz, nz)),\
        np.zeros((numBands, numBands, nz, nz))
    for ib1 in range(numBands):
        for ib2 in range(numBands):
            b1 = np.repeat(ib1, nz)
            b2 = np.repeat(ib2, nz)
            fz1 = 1 + zgrid
            fz2 = 1 + zgrid
            kernelparts(nz, nz, numCoefs, numLines,
                        alpha_C, alpha_L,
                        fcoefs_amp, fcoefs_mu, fcoefs_sig,
                        lines_mu[:numLines], lines_sig[:numLines], norms,
                        b1, fz1, b2, fz2, True,
                        KL_grid[ib1, ib2, :, :], KC_grid[ib1, ib2, :, :],
                        D_alpha_C_grid[ib1, ib2, :, :],
                        D_alpha_L_grid[ib1, ib2, :, :],
                        D_alpha_z_grid[ib1, ib2, :, :])

    Xrand = random_X_bzl(size, numBands=numBands)
    X2rand = random_X_bzl(size, numBands=numBands)
    NO1, NO2 = Xrand.shape[0], X2rand.shape[0]
    b1 = Xrand[:, 0].astype(int)
    b2 = X2rand[:, 0].astype(int)
    fz1 = 1 + Xrand[:, 1]
    fz2 = 1 + X2rand[:, 1]

    KC_rand, KL_rand =\
        np.zeros((NO1, NO2)),\
        np.zeros((NO1, NO2))
    D_alpha_C_rand, D_alpha_L_rand, D_alpha_z_rand =\
        np.zeros((NO1, NO2)),\
        np.zeros((NO1, NO2)),\
        np.zeros((NO1, NO2))
    kernelparts(NO1, NO2, numCoefs, numLines,
                alpha_C, alpha_L,
                fcoefs_amp, fcoefs_mu, fcoefs_sig,
                lines_mu[:numLines], lines_sig[:numLines], norms,
                b1, fz1, b2, fz2, True,
                KL_rand, KC_rand,
                D_alpha_C_rand, D_alpha_L_rand, D_alpha_z_rand)

    p1s = np.zeros(size, dtype=int)
    p2s = np.zeros(size, dtype=int)
    find_positions(size, nz, fz1, p1s, opzgrid)
    find_positions(size, nz, fz2, p2s, opzgrid)

    KC_interp, KL_interp =\
        np.zeros((NO1, NO2)),\
        np.zeros((NO1, NO2))
    KC_diag_interp, KL_diag_interp =\
        np.zeros((NO1, )),\
        np.zeros((NO1, ))
    D_alpha_C_interp, D_alpha_L_interp, D_alpha_z_interp =\
        np.zeros((NO1, NO2)),\
        np.zeros((NO1, NO2)),\
        np.zeros((NO1, NO2))

    kernel_parts_interp(size, size,
                        KC_interp,
                        b1, fz1, p1s,
                        b2, fz2, p2s,
                        opzgrid, KC_grid)
    print(np.abs(KC_interp/KC_rand - 1))
    assert np.mean(np.abs(KC_interp/KC_rand - 1)) < relative_accuracy
    assert np.max(np.abs(KC_interp/KC_rand - 1)) < relative_accuracy

    kernel_parts_interp(size, size,
                        D_alpha_C_interp,
                        b1, fz1, p1s,
                        b2, fz2, p2s,
                        opzgrid, D_alpha_C_grid)
    print(np.abs(D_alpha_C_interp/D_alpha_C_rand - 1))
    assert np.mean(np.abs(D_alpha_C_interp/D_alpha_C_rand - 1))\
        < relative_accuracy
    assert np.max(np.abs(D_alpha_C_interp/D_alpha_C_rand - 1))\
        < relative_accuracy


In [10]:
test_kernel_parts_interp()

[[8.97092702e-05 3.67034278e-06 3.63348520e-05 ... 5.31899541e-05
  8.09036888e-05 4.86043500e-05]
 [9.38291995e-05 1.20232293e-05 2.05555107e-05 ... 3.63594716e-05
  1.10470151e-04 7.85785526e-05]
 [6.28121503e-05 4.59175152e-05 1.07162273e-05 ... 3.71324445e-06
  8.50594723e-05 5.87258104e-05]
 ...
 [3.18911701e-05 2.69580782e-05 1.81002369e-05 ... 2.93615419e-05
  2.61134316e-05 1.03537812e-05]
 [1.30294820e-05 3.17806392e-04 3.27572839e-05 ... 2.14115580e-05
  9.43893851e-05 1.22967635e-04]
 [4.69498654e-05 1.09548058e-06 7.94204093e-05 ... 7.61731337e-05
  7.38831195e-05 5.63076021e-05]]
[[1.56976096e-04 1.62558467e-04 4.16112652e-05 ... 4.71222327e-05
  6.63992333e-05 9.52424328e-05]
 [1.89668031e-04 3.39921728e-04 7.03846472e-05 ... 3.59605177e-05
  9.22450511e-05 2.17642296e-05]
 [9.84252759e-05 2.07159296e-04 1.75574096e-06 ... 1.62806643e-05
  4.63706600e-05 6.15905196e-05]
 ...
 [8.86363469e-05 3.58799631e-06 9.34995430e-05 ... 5.29337525e-05
  6.46187560e-05 2.93401764e-05]